In [45]:
from numpy import *
from scipy.optimize import fsolve
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
%matplotlib inline

The first order catalytic reaction
$$A \rightarrow B$$
occurs in a cylindrical packed bed that consists out of solid catalyst (completely non porous
– only external surface reaction). An isothermal run gives a steady state conversion of
$46.6\%$. The following information is available on the reactor:

<img src="4.3a.JPG" width="450" />

In [46]:
#first order
#PBR
#non-porous catalyst - SOLID - ki will change with dp
#isothermal with steady state conversion

In [47]:
X       =      0.466 #conversion
W       =       0.15 #kg
Q       = 50e-3/3600 #m3/s
dp      =     2.5e-3 #m
kiprime =       3e-4 #m3/(kg.s)
rhobed  =       1200 #kg/m3
phi     =        0.4 #porosity

**a) What would the conversion be if external mass transfer was assumed to be
negligible?** [$X = 0.961$]

This question can easily be done analytically quite quickly:
\begin{align}
\dfrac{dF_{A}}{dW} &= r_{A}^{'} \nonumber \\
-QC_{A_{0}}\dfrac{dX}{dW} &= -k_{i}^{'}C_{A} \nonumber \\
\dfrac{dX}{dW} &= k_{i}^{'}\dfrac{C_{A_{0}}(1 - X)}{QC_{A_{0}}} \nonumber \\
\dfrac{dX}{dW} &= k_{i}^{'}\dfrac{(1 - X)}{Q} \nonumber \\
\int^{X}_{0}\dfrac{dx}{1-x}&=\dfrac{k_{i}^{'}W}{Q} \nonumber \\
-\ln(1-X)\big\rvert^{X}_0 &=\dfrac{k_{i}^{'}W}{Q}\nonumber \\
-\ln(1-X) &=\dfrac{k_{i}^{'}W}{Q}\nonumber \\
X &= 1 - e^{\dfrac{-k_{i}^{'}W}{Q}}\nonumber
\end{align}

In [48]:
Xa = 1 - exp((-kiprime*W)/Q)
print('Conversion= '+str(Xa*100)+'%')

Conversion= 96.0836104901013%


**b) What is the specific mass transfer coefficient (units m/s) in the reactor?** [$k_{c}= 6 \times 10^{-5}~m/s$]

The external catalyst surface area per $kg$ catalyst:
\begin{align}
a_{m} &= \dfrac{A_{sphere}}{V_{sphere}}\dfrac{1}{\rho_{cat}} \nonumber \\
&= \dfrac{6}{d_{p}}\dfrac{1}{\rho_{cat}} \nonumber \\
but\ \rho_{B} &= \rho_{cat}(1 - \epsilon_{Bed}) \nonumber \\
\therefore a_{m} &= \dfrac{6}{d_{p}}\dfrac{(1 - \epsilon)}{\rho_{Bed}} \nonumber
\end{align}

In [49]:
am = (6*(1 - phi))/(dp*rhobed)
am

1.2

We know what the conversion is and that external mass transfer plays a role, therefore we can modify the above equation from a) to calculate the effective constant:
\begin{align}
-\ln(1-X) &=\dfrac{k_{eff}^{'}W}{Q}\nonumber \\
k_{eff}^{'} &= -\dfrac{Q}{W}\ln(1-X)\nonumber
\end{align}

In [50]:
keff = -Q*log(1 - X)/W
keff

5.8088837039068724e-05

Because we are dealing with a first order reaction we can use the simplified equation:
\begin{align}
\dfrac{1}{k_{eff}^{'}} &= \dfrac{1}{k_{c}a_{m}} + \dfrac{1}{k_{i}^{'}}
\end{align}

In [51]:
p = (1/keff) - (1/kiprime)
kc = 1/(am*p)
kc

6.003116632576613e-05

-------------------------------------------------------------------------

*Given a new reactor with a different shape (also cylindrical) containing $150~g$ of $1.8~mm$(diameter) particles a $44.8~\%$ conversion was obtained. The active site surface concentration (i.e. (Atoms of catalytically active material)/area) for the $1.8~mm$ catalyst is double that of the $2.5~mm$ catalysts (same type of active site). Everything else in the given table remains the same.*

*Use the Theones Kramer correlation for the dependency of $k_{c}$ on different process and catalyst variables*

In [52]:
dp2 = 1.8e-3 #m
X2  = 0.448
#L1 = 2*L original

**c) Determine the mass based intrinsic constant of the 1.8 mm catalyst.** [$k_{i}^{'} = 8.33 \times 10^{-4}~m^{3}/(kg.s)$] (Note: function of particle diameter and loading density!)

Thoenes Kramer correlation to calculate the new $k_{c}$ value.

\begin{align}
\left[\frac{k_cd_p}{D_{AB}}\left(\frac{\epsilon}{1-\epsilon}\right)\frac{1}{\gamma}\right] &= \left[\frac{Ud_p\rho_L}{\mu(1-\epsilon)\gamma}\right]^{1/2}\left(\frac{\mu}{\rho D_{AB}}\right)^{1/3}\\
\end{align}

We know that:
\begin{align}
k_{i}^{'} &= k_{i}^{''}a_{m}L_{sites} \nonumber \\
k_{2}^{'} &= k_{i}^{''}a_{m2} 2\times L_{sites} \nonumber \\
\therefore k_{2}^{'} &= \dfrac{2a_{m2}}{a_{m}}k_{1}^{'} \nonumber
\end{align}

In [53]:
am2 = (6*(1 - phi))/(dp2*rhobed)
am2

1.6666666666666663

In [54]:
ki2 = 2*am2*kiprime/am
print('New ki= '+str(ki2*10000)+'10^-4')

New ki= 8.3333333333333310^-4


**d) Is the new reactor diameter bigger or smaller than the original one? What is the exact
ratio?** ($ 𝐷_{2} = 2 × 𝐷_{1}$)

In [55]:
keff2 = -Q*log(1 - X2)/W
keff2

5.501918821342978e-05

In [56]:
b = (1/keff2) - (1/ki2)
kc2 = 1/(am2*b)
kc2

3.534510361812439e-05

From the Thoenes Kramer correlation above, all else equal:
\begin{align}
\frac{k_{c2}}{k_{c1}}&=\left[\frac{U_{2}d_{p1}}{U_{1}d_{p2}}\right]^{1/2}\nonumber \\
&=\left[\frac{Q/A_{across2}}{Q/A_{across1}}\right]^{1/2}\left[\dfrac{d_{p1}}{d_{p2}}\right]^{1/2}\nonumber \\
&=\left[\frac{\pi D_{1}^{2}/4}{\pi D_{2}^{2}/4}\right]^{1/2}\left[\dfrac{d_{p1}}{d_{p2}}\right]^{1/2}\nonumber \\
\frac{k_{c2}}{k_{c1}} \left[\dfrac{d_{p2}}{d_{p1}}\right]^{1/2}&=\left[\frac{D_{1}}{D_{2}}\right]\nonumber 
\end{align}

In [57]:
Ratio = (kc2/kc)*((dp2/dp)**0.5)
Ratio

0.49959573960947135

Therefore $D_{2} = 2 \times D_{1}$